### Correcting Validity

In [14]:
"""
Your task is to check the "productionStartYear" of the DBPedia autos datafile for valid values.
The following things should be done:
- OK check if the field "productionStartYear" contains a year
- OK check if the year is in range 1886-2014
- OK convert the value of the field to be just a year (not full datetime)
- the rest of the fields and values should stay the same
- if the value of the field is a valid year in the range as described above,
  write that line to the output_good file
- if the value of the field is not a valid year as described above, 
  write that line to the output_bad file
- OK discard rows (neither write to good nor bad) if the URI is not from dbpedia.org
- you should use the provided way of reading and writing data (DictReader and DictWriter)
  They will take care of dealing with the header.

You can write helper functions for checking the data and writing the files, but we will call only the 
'process_file' with 3 arguments (inputfile, output_good, output_bad).
"""
import csv
import pprint
import re
from datetime import datetime

INPUT_FILE = 'autos.csv'
OUTPUT_GOOD = 'autos-valid.csv'
OUTPUT_BAD = 'FIXME-autos.csv'

def process_file(input_file, output_good, output_bad):
    with open(input_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames

        #COMPLETE THIS FUNCTION
        good_autos = []
        bad_autos  = []
        
        for row in reader:
            # discard rows if the URI is not from dbpedia.org
            if not row['URI'].startswith('http://dbpedia.org/'):
                continue
            
            prod_start_year = row['productionStartYear']
            # check if the field "productionStartYear" contains a year
            if prod_start_year == 'NULL' or not re.match('^[0-9]{4}.*', prod_start_year):
                bad_autos.append(row)
            else:
                prod_datetime = datetime.strptime(prod_start_year, '%Y-%m-%dT%H:%M:%S+02:00')
                
                # convert the value of the field to be just a year (not full datetime)
                prod_year = prod_datetime.year
                row['productionStartYear'] = prod_year
                
                # check if the year is in range 1886-2014
                if prod_year < 1886 or prod_year > 2014:
                    bad_autos.append(row)
                else:
                    good_autos.append(row)
        #print bad_autos
        write_output_file(output_bad, bad_autos)
        write_output_file(output_good, good_autos)

def write_output_file(filename, data):
    # This is just an example on how you can use csv.DictWriter
    # Remember that you have to output 2 files
    with open(filename, "w") as g:
        writer = csv.DictWriter(g, delimiter=",", fieldnames=data[0].keys())
        writer.writeheader()
        for row in data:
            writer.writerow(row)


def test():

    process_file(INPUT_FILE, OUTPUT_GOOD, OUTPUT_BAD)


if __name__ == "__main__":
    test()

### Auditing data quality

In [49]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up. In the first exercise we want you to audit
the datatypes that can be found in some particular fields in the dataset.
The possible types of values can be:
- NoneType if the value is a string "NULL" or an empty string ""
- list, if the value starts with "{"
- int, if the value can be cast to int
- float, if the value can be cast to float, but CANNOT be cast to int.
   For example, '3.23e+07' should be considered a float because it can be cast
   as float but int('3.23e+07') will throw a ValueError
- 'str', for all other values

The audit_file function should return a dictionary containing fieldnames and a 
SET of the types that can be found in the field. e.g.
{"field1": set([type(float()), type(int()), type(str())]),
 "field2": set([type(str())]),
  ....
}
The type() function returns a type object describing the argument given to the 
function. You can also use examples of objects to create type objects, e.g.
type(1.1) for a float: see the test function below for examples.

Note that the first three rows (after the header row) in the cities.csv file
are not actual data points. The contents of these rows should note be included
when processing data types. Be sure to include functionality in your code to
skip over or detect these rows.
"""
import codecs
import csv
import json
import pprint
import re

CITIES = 'cities.csv'

FIELDS = ["name", "timeZone_label", "utcOffset", "homepage", "governmentType_label",
          "isPartOf_label", "areaCode", "populationTotal", "elevation",
          "maximumElevation", "minimumElevation", "populationDensity",
          "wgs84_pos#lat", "wgs84_pos#long", "areaLand", "areaMetro", "areaUrban"]

def audit_file(filename, fields):
    fieldtypes = {key : set() for key in fields}
    
    # YOUR CODE HERE
    with open (filename, 'r') as f:
        reader = csv.DictReader(f)
        line_number = 0
        for row in reader:
            line_number += 1
            if line_number <= 4:
                continue
        
            for fld in fields:
                value = row[fld]
                tp = None
                try:
                    if value == "NULL" or value == "":
                        tp = type(None)
                    elif value.startswith("{"):
                        tp = type([])
                    elif re.match('^[0-9]+$', value):
                        tp = type(int(value))
                    else:
                        tp = type(float(value))
                except ValueError:
                    tp = type(str())
                    
                fieldtypes[fld].add(tp)

    return fieldtypes


def test():
    fieldtypes = audit_file(CITIES, FIELDS)

    pprint.pprint(fieldtypes)

    assert fieldtypes["areaLand"] == set([type(1.1), type([]), type(None)])
    assert fieldtypes['areaMetro'] == set([type(1.1), type(None)])
    
if __name__ == "__main__":
    test()

{'areaCode': set([<type 'int'>, <type 'NoneType'>, <type 'str'>]),
 'areaLand': set([<type 'float'>, <type 'list'>, <type 'NoneType'>]),
 'areaMetro': set([<type 'float'>, <type 'NoneType'>]),
 'areaUrban': set([<type 'float'>, <type 'NoneType'>]),
 'elevation': set([<type 'float'>, <type 'list'>, <type 'NoneType'>]),
 'governmentType_label': set([<type 'NoneType'>, <type 'str'>]),
 'homepage': set([<type 'NoneType'>, <type 'str'>]),
 'isPartOf_label': set([<type 'list'>, <type 'NoneType'>, <type 'str'>]),
 'maximumElevation': set([<type 'NoneType'>]),
 'minimumElevation': set([<type 'NoneType'>]),
 'name': set([<type 'list'>, <type 'NoneType'>, <type 'str'>]),
 'populationDensity': set([<type 'float'>, <type 'list'>, <type 'NoneType'>]),
 'populationTotal': set([<type 'int'>, <type 'NoneType'>]),
 'timeZone_label': set([<type 'NoneType'>, <type 'str'>]),
 'utcOffset': set([<type 'float'>,
                   <type 'list'>,
                   <type 'NoneType'>,
                   <type 

### Fixing the area

In [65]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.

Since in the previous quiz you made a decision on which value to keep for the
"areaLand" field, you now know what has to be done.

Finish the function fix_area(). It will receive a string as an input, and it
has to return a float representing the value of the area or None.
You have to change the function fix_area. You can use extra functions if you
like, but changes to process_file will not be taken into account.
The rest of the code is just an example on how this function can be used.
"""
import codecs
import csv
import json
import pprint

CITIES = 'cities.csv'


def fix_area(area):

    # YOUR CODE HERE
    if area == "NULL" or area == "":
        area = None
    elif area.startswith('{'):
        area = area[1:len(area)-1]
        area_values = str.split(area, '|')
        
        if len(area_values[0]) > len(area_values[1]):
            area = float(area_values[0])
        else:
            area = float(area_values[1])
    else:
        area = float(area)
        
    return area



def process_file(filename):
    # CHANGES TO THIS FUNCTION WILL BE IGNORED WHEN YOU SUBMIT THE EXERCISE
    data = []

    with open(filename, "r") as f:
        reader = csv.DictReader(f)

        #skipping the extra metadata
        for i in range(3):
            l = reader.next()

        # processing file
        for line in reader:
            # calling your function to fix the area value
            if "areaLand" in line:
                line["areaLand"] = fix_area(line["areaLand"])
            data.append(line)

    return data


def test():
    data = process_file(CITIES)

    print "Printing three example results:"
    for n in range(5,8):
        pprint.pprint(data[n]["areaLand"])

    assert data[3]["areaLand"] == None        
    assert data[8]["areaLand"] == 55166700.0
    assert data[20]["areaLand"] == 14581600.0
    assert data[33]["areaLand"] == 20564500.0    


if __name__ == "__main__":
    test()

Printing three example results:
None
101787000.0
31597900.0


### Fixing Name

In [68]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.

In the previous quiz you recognized that the "name" value can be an array (or
list in Python terms). It would make it easier to process and query the data
later if all values for the name are in a Python list, instead of being
just a string separated with special characters, like now.

Finish the function fix_name(). It will recieve a string as an input, and it
will return a list of all the names. If there is only one name, the list will
have only one item in it; if the name is "NULL", the list should be empty.
The rest of the code is just an example on how this function can be used.
"""
import codecs
import csv
import pprint

CITIES = 'cities.csv'


def fix_name(name):
    
    # YOUR CODE HERE
    if name == "NULL":
        name = []
    elif name.startswith("{"):
        name = name[1:len(name)-1]
        name = str.split(name, '|')
    else:
        name = [name]
    return name


def process_file(filename):
    data = []
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        #skipping the extra metadata
        for i in range(3):
            l = reader.next()
        # processing file
        for line in reader:
            # calling your function to fix the area value
            if "name" in line:
                line["name"] = fix_name(line["name"])
            data.append(line)
    return data


def test():
    data = process_file(CITIES)

    print "Printing 20 results:"
    for n in range(20):
        pprint.pprint(data[n]["name"])

    assert data[14]["name"] == ['Negtemiut', 'Nightmute']
    assert data[9]["name"] == ['Pell City Alabama']
    assert data[3]["name"] == ['Kumhari']

if __name__ == "__main__":
    test()

Printing 20 results:
['Kud']
['Kuju']
['Kumbhraj']
['Kumhari']
['Kunigal']
['Kurgunta']
['Athens']
['Demopolis']
['Chelsea Alabama']
['Pell City Alabama']
['City of Northport']
['Sand Point']
['Unalaska Alaska']
['City of Menlo Park']
['Negtemiut', 'Nightmute']
['Fairbanks Alaska']
['Homer']
['Ketchikan Alaska']
['Nuniaq', 'Old Harbor']
['Rainier Washington']


### Crossfield Auditing

In [71]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.

If you look at the full city data, you will notice that there are couple of
values that seem to provide the same information in different formats: "point"
seems to be the combination of "wgs84_pos#lat" and "wgs84_pos#long". However,
we do not know if that is the case and should check if they are equivalent.

Finish the function check_loc(). It will recieve 3 strings: first, the combined
value of "point" followed by the separate "wgs84_pos#" values. You have to
extract the lat and long values from the "point" argument and compare them to
the "wgs84_pos# values, returning True or False.

Note that you do not have to fix the values, only determine if they are
consistent. To fix them in this case you would need more information. Feel free
to discuss possible strategies for fixing this on the discussion forum.

The rest of the code is just an example on how this function can be used.
Changes to "process_file" function will not be taken into account for grading.
"""
import csv
import pprint

CITIES = 'cities.csv'


def check_loc(point, lat, longi):
    # YOUR CODE HERE
    point_values = str.split(point, ' ')
    point_lat = point_values[0]
    point_longi = point_values[1]
    
    if point_lat == lat and point_longi == longi:
        return True
    else:
        return False


def process_file(filename):
    data = []
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        #skipping the extra matadata
        for i in range(3):
            l = reader.next()
        # processing file
        for line in reader:
            # calling your function to check the location
            result = check_loc(line["point"], line["wgs84_pos#lat"], line["wgs84_pos#long"])
            if not result:
                print "{}: {} != {} {}".format(line["name"], line["point"], line["wgs84_pos#lat"], line["wgs84_pos#long"])
            data.append(line)

    return data


def test():
    assert check_loc("33.08 75.28", "33.08", "75.28") == True
    assert check_loc("44.57833333333333 -91.21833333333333", "44.5783", "-91.2183") == False

if __name__ == "__main__":
    test()